There is a a way to get the SqrtPriceX96 from the tick and vice-versa:

\begin{align*}
P &= 1.0001^{\text{tick}} \\
\sqrt{P} &= 1.0001^{\frac{\text{tick}}{2}} = \left(1.0001^{\text{tick}}\right)^{\frac{1}{2}} \\
\log_{\sqrt{1.0001}}\left(\sqrt{P}\right) &= \text{tick}
\end{align*}


Tick to SqrtPriceX96:


$$\sqrt{1.0001}^{\text{tick}} \times Q96 = \sqrt{P} \times Q96 = \text{sqrtPriceX96}$$


SqrtPriceX96 to Tick:

$$ \sqrt{P}  =  \frac{sqrtPriceX96}{Q96}  $$


$$ 

\begin{align*}
\log_{\sqrt{1.0001}}\left(\frac{sqrtPriceX96}{Q96}\right) &= \text{tick}
\end{align*}
$$



In [4]:
import math

Q96 = 2 ** 96

def tick_to_sqrt_price_x_96(tick):
    return int(1.0001 ** (tick / 2) * Q96)

def sqrt_price_x_96_to_tick(sqrt_price_x_96):
    base = math.sqrt(1.0001)
    p = sqrt_price_x_96 / Q96
    return math.floor(math.log(p, base))

tick = 204632

tick_to_sqrt_price_x_96(tick)

sqrt_price_x_96 = 2198835844819193856025769912483279
sqrt_price_x_96_to_tick(sqrt_price_x_96)


204632

We can see it works by using sample data of the substream:

In [7]:
sqrtPicex96 = 1523469420953906108383481136475199
# the outputted value of the tick on the vent
tick = 197293

sqrt_price_x_96_to_tick(sqrtPicex96)


197293

We are interested in calculating realized volatility using differing bases for our calculation. This would enable us to calculate volatility using either ticks or prices.

Give price $P_i$, let:
$$ u_i = \ln\frac{P_i}{P_{i-1}} = \ln\Delta P $$

The textbook definition of realized volatility is:
\begin{align*}
s &= \sqrt{\frac{1}{n-1} \sum_{i=1}^{n} (u_i-\bar{u})^2} \\
&= \sqrt{\frac{1}{n-1} \sum_{i=1}^{n} (u_i - \frac{1}{n}\sum_{i=1}^{n}u_i)^2}
\end{align*}

Suppose we wanted to use the tick values instead of the prices to calculate volatility. As shown above, the tick values are given by:

$$ tick_i = \log_{1.0001} P_i $$

We use the properties of logarithms to relate $\ln$ to $\log_{1.0001}$:

$$ \ln(x) = \log_{1.0001}(x) \cdot \ln(1.0001) $$
 
Using the above property we can relate the tick values to the $u_i$ calculation above:

\begin{align*}

u_i &= \ln(\frac{P_i}{P_{i-1}}) \\
&= \ln({P_i}) - \ln({P_{i-1}}) \\
&= \log_{1.0001}(P_i)\cdot\ln(1.0001) - \log_{1.0001}(P_{i-1})\cdot\ln(1.0001) \\
&= \ln(1.0001)\cdot(\log_{1.0001}(P_i) - \log_{1.0001}(P_{i-1})) \\
&= \ln(1.0001)\cdot(tick_i - tick_{i-1})

\end{align*}

Substituting this value into the equation for volatility above, we have:

\begin{align*}
s &= \sqrt{\frac{1}{n-1} \sum_{i=1}^{n} \left(u_i - \frac{1}{n}\sum_{i=1}^{n}u_i\right)^2} \\
&= \sqrt{\frac{1}{n-1} \sum_{i=1}^{n} \left(\ln(1.0001)\cdot(tick_i - tick_{i-1}) - \frac{1}{n}\sum_{i=1}^{n}\ln(1.0001)\cdot(tick_i - tick_{i-1})\right)^2}
\end{align*}

and after some algebra:

\begin{align*}
&= \sqrt{\frac{1}{n-1} \sum_{i=1}^{n} \left(\ln(1.0001)\cdot(tick_i - tick_{i-1}) - \frac{\ln(1.0001)}{n}\sum_{i=1}^{n}(tick_i - tick_{i-1})\right)^2} \\
&= \sqrt{\frac{1}{n-1} \sum_{i=1}^{n} \ln(1.0001)^2\cdot\left((tick_i - tick_{i-1}) - \frac{1}{n}\sum_{i=1}^{n}(tick_i - tick_{i-1})\right)^2} \\
&= \sqrt{\frac{\ln(1.0001)^2}{n-1} \sum_{i=1}^{n} \left((tick_i - tick_{i-1}) - \frac{1}{n}\sum_{i=1}^{n}(tick_i - tick_{i-1})\right)^2} \\
&= \ln(1.0001)\cdot\sqrt{\frac{1}{n-1} \sum_{i=1}^{n} \left((tick_i - tick_{i-1}) - \frac{1}{n}\sum_{i=1}^{n}(tick_i - tick_{i-1})\right)^2} \\
\end{align*}

We see that calculating volatitily using the tick values is equivalent to calculating volatility using the prices. The primary difference is the factor of $\ln(1.0001)$